# CNN for Medical Image Classification

**Welcome to the CNN tutorial for Chest X-Ray Image Classification!**

This tutorial is based on the article written by Abhinav Saga for *Towards Data Science*. The original article can be found <a href="https://towardsdatascience.com/deep-learning-for-detecting-pneumonia-from-x-ray-images-fc9a3d9fdba8">here</a>.

This tutorial requires data hosted on Kaggle.com (<a href="https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia">source</a>). 

**You will need to create a verified Kaggle account, download the data and change the source directory in this file.**

Let's get started!

## Importing Packages and Dependencies
To get started, we need to import the required packages which the code will use. The architecture of the neural network will use Keras. 

Keras is a high level machine learning API which uses TensorFlow as a backend. This allows users to focus on implementation from a more blackbox perspective. TensorFlow is much better suited for more whitebox approaches. TensorFlow uses a structure called tensors which a basically n-dimensional arrays or matricies. We can use NumPy arrays to pass to TensorFlow and Keras.

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
import keras.backend as k
import tensorflow as tf

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

## Importing Images
We now need to set an input path from which to load images. **You must change the following line to point to the correct folder on your system.**

In [ ]:
inPath = "C:/Datasets/ChestXRay/chest_xray/chest_xray"

## Testing the Filepath
You can verify that the path has been set correctly by counting the images from the dataset.

The for loop below simply counts and displays the number of images in each directory. This gives us an idea of how many images we are dealing with in each set. You should confirm that your output is as follows:<br />
`Set: /train` <br />
`Normal Images: 1342 ` <br />
`Pneumonia Images: 3876` <br />
<br />
`Set: /val` <br />
`Normal Images: 9` <br />
`Pneumonia Images: 9` <br />
<br />
`Set: /test` <br />
`Normal Images: 234` <br />
`Pneumonia Images: 390` <br />

In [ ]:
for subset in ['/train', '/val', '/test']:
    normal = len(os.listdir(inPath + subset + "/NORMAL"))
    pneumonia = len(os.listdir(inPath + subset + "/PNEUMONIA"))
    print("Set: " + str(subset) + "\nNormal Images: " + str(normal) + "\nPneumonia Images: " + str(pneumonia) + "\n~~~~~~~~")

## Preparing the Data
Now that we have confirmed that we have access to the images, we need to process it into sets to that it be passed to the CNN. Abhinav does this by splitting the the images into training, validation and test sets. They also implement a technique called data augmentation. Because deep learning algorithms perform better with more training data, data augmentation is used to increase the size of the dataset used to train the model. This is relatively easy and a dataset size can be quadrupled simply by rotating images 90, 180 and 270 degrees though other methods also exist like flipping pictures. <br />
During this next step, a function is defined which augments and processes image data to prepare it to be passed to the model. The data is augmented using a Keras class called ImageDataGenerator which augments imagesets in real-time with the flow_from_directory() method.<br />
The function also resizes images and prepares them for the model.
<br />
<br /> Abhinav uses the following code:

In [ ]:
def process_data(img_dims, batch_size):
    # Data generation objects
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    input_path = inPath
    
    # This is fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'/train', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(
    directory=input_path+'/test', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)
    
    # I will be making predictions off of the test set in one batch size
    # This is useful to be able to get the confusion matrix
    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + '/test' + cond)):
            img = plt.imread(input_path+'/test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    return train_gen, test_gen, test_data, test_labels

## Hyperparameters
Hyperparameters are parameters that the creator of the algorithm must choose and directly affect model training accuracy. <br />
The `img_dims` variable represents the size of the image in pixels that the images will be resized to. Having a larger image size will drastically increase training 
time is set too high. Too small dimensions, however, results in poor training because information which which to learn is lost from the image when resized. <br />
The `epochs` variable represents the number of times the model runs through the complete training set. Too many epochs result in long wait times and may actually reduce the accuracy of the
learned model. Too little epochs and the model will not learn enough. <br/>
Finally, the `batch_size` variable is that controls the number of training samples to work through before a models internal parameters (like weights) are updated. Having too large a batch size will use a lot more resources and increase training time.<br />
<br />
The `train_gen`, `test_gen`, `test_data` and `test_labels` variables represent the training and test datasets which will be used to train the model. 
These variables are set below. We will experiment with different hyperparameters toward the end of this tutorial. For now the defaults remain: <br/>
`img_dims = 150
epochs = 20
batch_size = 32`

In [ ]:
img_dims = 150
my_epochs = 20
batch_size = 32

train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)

## Defining the CNN Architecture
The most important (and fun) step! Keras makes creating neural networks extremely intuitive and easy. In fact, it may seem like all the previous steps are the most difficult to understand initially. <br />
To generate this CNN, we must first contruct each layer in the neural network. We'll walk through Abhinav's CNN layer by layer.<br/>
But, before we get started, we need to create an input tensor (a matrix) that is the dimensions of the images being used to train the neural network and an added third dimension of 3. This third dimension is due to the three channels used to compose JPEG images (height x width x channel) which are actually three dimensional array data structures!

In [ ]:
inputs = Input(shape=(img_dims, img_dims, 3))

Now we create the first layers in the CNN, which takes `inputs`, the JPEG images, as an input. The convolutional layer scans a kernel across the image and produces another tensor which maps the activation of the kernel across the image. This is the first step in feature extraction. The resulting tensor created from this convolutional layer is then passed to another convolutional layer, and the tensor output by this layer is passed to a maxpooling layer that reduces dimensionality of the data and helps the network to learn and extract features from the image produced the convolution. This completes the first convolutional block.

In [ ]:
# first conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

In [ ]:
# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

Finally, an FC layer is present which classifies the features extracted by the previous convolutional layers and passes this information to the output layer.


In [ ]:
# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=1, activation='sigmoid')(x)

## Compiling The Model
Now, we can compile the model. But before we do that, there are some things to consider.<br/>
1. Training can take a very long time, from hours possibly to days <br/>
2. If training is interupted, then progress is lost <br/>
3. We can't know the what the perfect number of epochs is that is high enough to prevent underfitting, but low enough to prevent overfitting 


In order to prevent losing a model during training, we can implement the `ModelCheckpoint` callback. This call back saves the model after every epoch to a file at specified `filepath`. If the `save_best_only` boolean is set to `True` then the model will save only the best epochs. The `save_weights_only` boolean means that instead of saving all parameters in the model, just the weights of each connection are saved. <br/>
<br/>
Choosing hyperparameters like epoch, can be done using trial and error, but this is tedious and may cause overfitting if the model is allowed to train for too many epochs. In order to prevent overfitting, we can used both the `ReduceLROnPlateau` and `EarlyStopping` callbacks. The the `ReduceLROnPLateau` callback method reduces the learning rate of the model once learning starts to plateau. The `EarlyStopping` callback stops the training process when a certain metric that is being monitored stops improving.<br/><br/>
For `ReduceLROnPlateau`, the value which we'd like to monitor is the model's loss, so `monitor` is set to `val_loss`. The factor value is factor by which the learning rate will be reduced. `patience` is the number of consecutive epochs with no improvements to wait before reducing learning rate, while `verbose` simply provides a warning message saying that the learning rate is being reduced. Finally, `mode` tells the method to look for improvements in increasing loss when set to `'max'`. When set to `'min'`, the method looks for decreasing loss. In total, this method is saying, monitor`val_loss` as it approaches some `max`imum. If the loss fails to increase for `2` consecutive epochs, reduce the learning rate by a factor of `0.3` and warn the user.<br/><br/>
For `EarlyStopping`, the value being monitor is again `val_loss`, just as in the previous method. There is also another parameter, `min_delta` which is set to `0.1`. This parameter specifies the minimum change in the monitored value to be concidered an improvement. `mode` is set to `min` this time because we want the method to detect when `val_loss` reduces. Finally, the `patience` parameter is set to 3 so as to not interfere with `ReduceLSOnPlateau`.
<br/>
<br/>
To read more about the <a href="https://keras.io/api/callbacks/reduce_lr_on_plateau/">`ReduceLROnPlateau`</a> and <a href="https://keras.io/api/callbacks/early_stopping/">`EarlyStopping`</a> methods (actually they are classes), then see the appropriate documentation by clicking them.


In [ ]:
# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='xray_model.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=3, mode='min')

my_callbacks = [
    checkpoint,
    lr_reduce,
    early_stop
]

## Model Training
Finally, we can train the model! 
<br/>
The `model.fit()` method trains the model and stores training history to help visualize and track the training process. 
<br/>
We're using a few parameters here with `model.fit()`. The first, `train_gen`, passes the training set generator which was definined above with the line: <br/>
`train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)`.<br/> 
Next, the `epochs` parameter sets the number of training runs, epochs, to perform training on. We defined `my_epochs` above to be 100. The `callbacks` parameter is used to set the callbacks we want the program to use during training. These were diffined above as `my_callbacks`. 

In [ ]:
hist = model.fit(
            train_gen, 
            epochs=my_epochs,
            callbacks=my_callbacks,
            # verbose = 1 # Default
            validation_data=test_gen, 
            steps_per_epoch=train_gen.samples // batch_size,
            validation_steps=test_gen.samples // batch_size
)

Using the history, we can create loss and accuracy plots to visualize model training.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

We can also gather more useful data like accuracy and pricision by calculating a confusion matrix and other test metrics. Abhinav uses the following code.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))

## Optional: Experiment with Hyperparameters
Now you can edit the hyperparametes above and re-run the proceeding steps again to observe any changes. <br/>What changes increase training time?<br/> What changes increase accuracy?<br/> Can you outperform your initial result by altering these parameters?

## Saving Models
So you've trained the model, great. But we want to actually use the model that we've trained, otherwise, what good is it?
<br/>
Luckily, TensorFlow and Keras make it very easy to save a complete model for further use and/or further training later.
<br/>
This can be done very easily.

In [ ]:
## model.save('path/to/file/filename')
model.save('C:/Models/CNNexample')

## Loading Models
Loading a saved model is just as easy.

In [ ]:
from tensorflow import keras

In [ ]:
## model = keras.models.load_model('path/to/file/filename')
model = keras.models.load_model('C:/Models/CNNexample')

## Using a Trained Model
Now we can use a trained model (that was loaded, or that you just trained) to make a prediction based on your data. 

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = 'person19_virus_50.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

prediction = model.predict(x)

print(prediction)

If `[[0.]]` was printed, the model misclassfied the image we passed it. Let's try training it some more.

In [ ]:
hist = model.fit_generator(
           train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=20, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

Let's save the new model...

In [ ]:
## model.save('path/to/file/filename')
model.save('CNNexample')

## New Prediction
Now, we'll try another prediction. Lets compare predictions that this CNN will make for a normal picture and a picture with pneumonia.

In [ ]:
## Pneumonia X-ray should score closely to 1
img_path = 'pneumonia.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
pred = image.img_to_array(img)
pred = np.expand_dims(pred, axis=0)
pred = pred.astype('float32')/255

prediction = model.predict(pred)

print(prediction)

In [ ]:
## Normal X-ray should score closely to 0
img_path = 'normal.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
pred = image.img_to_array(img)
pred = np.expand_dims(pred, axis=0)
pred = pred.astype('float32')/255

prediction = model.predict(pred)

print(prediction)

## DONE!
Now that you've completed the tutorial, you should go through and experiment with different hyperparameters. <br/><br/>
I would recommend considering the following:
1. Should both `ReduceLROnPlateau` and `EarlyStopping` be used?
2. If both are used, what parameters should be changed to ensure they don't interfere with eachother and still allow the model to train? (hit, you should have at least 10-15 epochs for this particular model with this data to be trained effectively.
3. How does adjusting batch size affect learning and the speed of learning?
4. What structural changes can be made to the CNN that could make it perform better? (I would definitely exeperiment here with different CNN architectures!)
5. After you've experimented with this data, try to modify to the code to use another dataset of your choice. Importantly, if you do so, you must ensure that the images are atleast 150x150 px in size.
6. Try changing the CNN to take a larger image size. Does is take longer to train? Is it more accurate?